In [51]:
import tdmclient.notebook
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning
import obstacle_avoidance

In [52]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def get_prox():
    global prox_horizontal
    obst = prox_horizontal
    return obst

In [87]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""


TRESH_DIST = 20 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436 #150 speed
YAW_TO_MMS = 0.06086 # -100 100
SPEED_OFFSET = 5

def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        # we need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        print("Px_to_mm= ", px_to_mm)
        
        px_to_mm = 0.64
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_obstacle.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_path.jpg', frame_init) #POUR TUNING PAR ELIOTT

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame.
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        # x_est_t_minus_1 = np.array([0.0,0.0,0.0])
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[0.1,  0,   0],
                                [  0,0.1,   0],
                                [  0,  0, 0.1]])
    
    
        #update le dt car sinon kalman marche pas
        dt = 0.1 # EN SECONDES POUR KALMAN!!!!! same as PI_CLOCK of pid defined in filters?
        sum_error = 0 
        alt_error_pid = 0
        v_l = 0
        v_r = 0
        
        state = 0
        obstThrH = 1400
        
        time.sleep(0.1)
        
        while len(goal_list)!=0:
            
            #print("goal is: ", goal_list[0])
            
            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, is_from_camera = True)
            
            
#             print(f'Timestep measurement={obs_vector_z_t}')
    
            # Run the Extended Kalman Filter and store the 
            # near-optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.ekf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Time interval
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, is_from_camera = False)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, is_from_camera = 2)
            
            frame = cv2.resize(frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
                
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            #check if obstacle in coming
            obst=get_prox()
            
            if state == 0:
                for i in range(len(obst)-2):
            # switch from goal tracking to obst avoidance if obstacle detected
                    if (obst[i] > obstThrH):
                        print('obst', obst)
                        state = 1
            if state==0:
                sum_error, alt_error_pid, v_l,v_r = filters.pid(optimal_state_estimate_t,goal_list[0],sum_error, alt_error_pid,dt)
    
            else:  #if yes, obstacle avoidance
                v, state = obstacle_avoidance.obstacle_avoidance(obst, v_l, v_r, True)
                v_l = v[0]
                v_r = v[1]
            
            v = (v_l + v_r)*SPEED_TO_MMS/2*1.1
            yaw = (v_l-v_r)*YAW_TO_MMS*4
            u_t_minus_1 = [v, yaw]; # delta_v returned by PID
                
            set_speed(int(v_l), int(v_r+SPEED_OFFSET))

            actual = time.time()
            diff = actual-previous
#             print("elapsed time is: ", diff)
            if diff<dt:
                time.sleep(dt-diff)
            
    finally: 
        set_speed(0, 0)
        cv2.destroyAllWindows()



In [54]:
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [57]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [58]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [92]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)
importlib.reload(obstacle_avoidance)

<module 'obstacle_avoidance' from 'C:\\Users\\Juliette\\Documents\\_MobileRobotics\\Mobile_robotics\\obstacle_avoidance.py'>

In [93]:
main()

take the rigth image
Px_to_mm=  0.46550960665762725


100%|██████████| 1/1 [00:00<00:00, 39.36it/s]


angle du robot 30.96375653207352
angle du robot 29.622643949213693
angle du robot 28.50695270518591
angle du robot 32.64321608558518
angle du robot 36.376706683054685
angle du robot 38.189673481746205
angle du robot 42.5645992405916
angle du robot 45.0285073456275
angle du robot 47.57264721891688
angle du robot 49.181201972921464
angle du robot 50.06510870536146
angle du robot 53.04427543317987
angle du robot 55.187127550287684
angle du robot 57.007426084629316
angle du robot 58.35167522723802
angle du robot 59.291328072772956
angle du robot 61.01939864907431
angle du robot 65.93592573684683
angle du robot 65.0348434252523
angle du robot 80.74873167752932
angle du robot 69.06265361336563
angle du robot 68.85652693080307
angle du robot 67.61371432098329
angle du robot 67.29091170823042
angle du robot 69.29435376550121
angle du robot 69.44599151354596
angle du robot 69.6713009710066
angle du robot 68.5570636058571
angle du robot 68.90581607238927
angle du robot 70.77761250470947
angle du

angle du robot 243.34567655558143
angle du robot 238.4276193614602
angle du robot 234.37843362188914
angle du robot 77.17897327342416
angle du robot 231.9269411442836
angle du robot 224.79653845228623
angle du robot 222.64825931303585
angle du robot 219.5212793925587
angle du robot 216.66014731431042
angle du robot 215.95303073482077
angle du robot 213.2086035109673
angle du robot 211.52036540094437
angle du robot 209.99139844174212
obst [1859, 0, 0, 0, 0, 0, 0]
remanent left speed:  -14.172714761162723
remanent right speed:  14.172714761162723
left speed:  -14.172714761162723
right speed:  14.172714761162723
angle du robot 207.27234237965013
obst [1859, 0, 0, 0, 0, 0, 0]
remanent left speed:  42.0
remanent right speed:  26.0
left speed:  42.0
right speed:  26.0
angle du robot 207.22534992913774
obst [2752, 0, 0, 0, 0, 0, 0]
remanent left speed:  70.0
remanent right speed:  32.0
left speed:  70.0
right speed:  32.0
angle du robot 209.07549392913774
remanent left speed:  91.0
remanent r

angle du robot 310.8330757635903
remanent left speed:  40.5
remanent right speed:  92.5
left speed:  40.5
right speed:  92.5
angle du robot 312.305379281345
remanent left speed:  55.0
remanent right speed:  81.0
left speed:  55.0
right speed:  81.0
angle du robot 311.7313997344244
remanent left speed:  62.5
remanent right speed:  75.5
left speed:  62.5
right speed:  75.5
angle du robot 311.09845573442436
remanent left speed:  66.0
remanent right speed:  72.5
left speed:  66.0
right speed:  72.5
angle du robot 310.3533650515054
remanent left speed:  68.0
remanent right speed:  71.0
left speed:  68.0
right speed:  71.0
angle du robot 310.7009421049628
remanent left speed:  69.0
remanent right speed:  70.5
left speed:  69.0
right speed:  70.5
angle du robot 310.4093706399135
angle du robot 310.3850266399135
angle du robot 312.3504916032598
angle du robot 308.29693555069287
angle du robot 312.9331607628937
angle du robot 314.78765062958
angle du robot 316.4615576910079
angle du robot 316.5

IndexError: list index out of range

In [50]:
cap.release()

In [29]:
await tdmclient.notebook.stop()